* Для работы 1 раз запустить первый и второй блок. Запускать блок можно, нажав на него мышкой, а затем нажав Cell->Run в верхнем меню.
* Далее работаем только в блоке 3, стирая символ '#' у необходимых в данный момент строк.
* В файле 1.json каходятся настройки программы. Обязательно поменять значение ключа 'path', на путь к папке landsat. Изменение остальных настроек опционально.

In [1]:
%load_ext Cython

In [154]:
%%cython
import os
import simplejson
import numpy as np
cimport numpy as np
cimport cython
import skimage.io
import matplotlib

# class for one scene analyse
cdef class Scene:
    cdef public int id, bit
    cdef object json, path
    
    def __init__(self, _id_, config_file='2.json'):
        self.id = _id_
        with open(config_file) as data_file:    
            self.json = simplejson.load(data_file)
        self.bit = int(self.json['scenes'][self.id]['bit'])
        self.path = self.json['scenes'][self.id]['path']
    
    def read_images(self, path):
        return {'red': skimage.io.imread(os.path.join(self.json['path'], self.json['paths'][path], 'red',
                                                     self.path +  self.json['suffs'][path])),
                'nir': skimage.io.imread(os.path.join(self.json['path'], self.json['paths'][path], 'nir',
                                                     self.path +  self.json['suffs'][path]))}
    
    cdef is_in_mask(self, unsigned short val):
        return val < 64000 if self.bit == 16 else val < 253
    
    def normalize_imgs(self, img_red, img_nir):
        arr_n = np.genfromtxt(self.json['docs']['n'])[self.id]
        img_red_n = (np.copy(img_red.astype(np.float)) - arr_n[0]) / arr_n[1]
        img_nir_n = (np.copy(img_nir.astype(np.float)) - arr_n[2]) / arr_n[3]
        img_red_n[img_red <= 0] = -99
        img_nir_n[img_red <= 0] = -99
        return img_red_n, img_nir_n
        
    def get_plot_array(self, imgs):
        cdef np.ndarray[np.uint16_t, ndim = 2] img_red = imgs['red'].astype('uint16', copy = False)
        cdef np.ndarray[np.uint16_t, ndim = 2] img_nir = imgs['nir'].astype('uint16', copy = False)
        sizes = self.json['plotSizes'][str(self.bit)]
        cdef int sn0 = sizes['nir'][0], sn1 = sizes['nir'][1], sr0 = sizes['red'][0], sr1 = sizes['red'][1]
        cdef np.ndarray[np.int32_t, ndim = 2] plot_array = np.zeros([sr1 - sr0 + 1, sn1 - sn0 + 1], dtype = 'int32')
        cdef int x, y
        for x in range(img_red.shape[1]):
            for y in range(img_red.shape[0]):
                if self.is_in_mask(img_red[y, x]) and self.is_in_mask(img_nir[y, x]) \
                    and sr0 <= img_red[y, x] <= sr1 and sn0 <= img_nir[y, x] <= sn1:
                        plot_array[img_nir[y, x] - sn0, img_red[y, x] - sr0] += 1
        return plot_array
    
    def save_plot_array(self, plot_array):
        matplotlib.image.imsave(os.path.join(self.json['path'], self.json['paths']['notMarkedPlots'], 
                                             self.path + self.json['suffs']['markedPlots']), 
                                plot_array[::-1, :])
    def make_solp_scene(self):
        imgs = self.read_images('inFiltered')
        cdef np.ndarray[np.uint16_t, ndim = 2] img_red = imgs['red'].astype('uint16', copy = False)
        cdef np.ndarray[np.uint16_t, ndim = 2] img_nir = imgs['nir'].astype('uint16', copy = False)
        sizes = self.json['plotSizes'][str(self.bit)]
        cdef int sn0 = sizes['nir'][0], sn1 = sizes['nir'][1], sr0 = sizes['red'][0], sr1 = sizes['red'][1]
        cdef np.ndarray[np.uint8_t, ndim = 3] plot_marked = skimage.io.imread(
            os.path.join(self.json['path'], self.json['paths']['markedPlots'], 
                         self.path + self.json['suffs']['markedPlots']))
        cdef np.ndarray[np.uint8_t, ndim = 2] colors = \
            np.array(self.json['colorsMarkedPlots']).astype('uint8', copy = False)
        cdef int x, y, need_white, red, nir, color_eps = 9, color_sum, c, cs
        for x in range(img_red.shape[1]):
            for y in range(img_red.shape[0]):
                red, nir = img_red[y, x], img_nir[y, x]
                need_white = 1
                if sr0 <= red <= sr1 and sn0 <= nir <= sn1:
                    for c in range(colors.shape[0]):
                        color_sum = 0
                        for cs in range(3):
                            color_sum += abs(colors[c, cs] - plot_marked[sn1 - nir - 1, red - sr0, cs]) # TODO -1
                        if (color_sum <= color_eps):
                            need_white = 0
                            break
                if need_white == 1:
                    img_nir[y, x] = img_red[y, x] = 65535 # if self.bit == 16 else 255
        for img in [[img_red, 'red'], [img_nir, 'nir']]:
            skimage.io.imsave(os.path.join(self.json['path'], self.json['paths']['solp'], img[1], 
                                           self.path + self.json['suffs']['solp']), 
                              img[0].astype('uint' + str(self.bit)), plugin = 'tifffile')
    
    def get_soil_line(self, imgs):
        cdef np.ndarray[np.uint16_t, ndim = 2] img_red = imgs['red'].astype('uint16', copy = False)
        cdef np.ndarray[np.uint16_t, ndim = 2] img_nir = imgs['nir'].astype('uint16', copy = False)
        cdef np.ndarray[np.uint16_t, ndim = 1] red = np.zeros([img_red.size], dtype = 'uint16')
        cdef np.ndarray[np.uint16_t, ndim = 1] nir = np.zeros([img_red.size], dtype = 'uint16')
        cdef int x, y, size = 0
        for x in range(img_red.shape[1]):
            for y in range(img_red.shape[0]):
                if self.is_in_mask(img_red[y, x]) and self.is_in_mask(img_nir[y, x]):
                    red[size] = img_red[y, x]
                    nir[size] = img_nir[y, x]
                    size += 1
        red = np.resize(red, size)
        nir = np.resize(nir, size)
        return np.linalg.lstsq(np.vstack([red, np.ones(len(red))]).T, nir)[0]
                    
    cdef brat_vec(self, np.ndarray[np.uint16_t, ndim = 2] img_red, np.ndarray[np.uint16_t, ndim = 2] img_nir,
        int pnt_x, int pnt_y, int rad, float line_a, float line_b):
        cdef float res_x = 0, res_y = 0, ave_x = 0, ave_y = 0, ave_subj = 0
        cdef int n = 0, x, y
        for y in xrange(max(pnt_y - rad, 0), min(pnt_y + rad, img_red.shape[0])):
            for x in xrange(max(pnt_x - rad, 0), min(pnt_x + rad, img_red.shape[1])):
                if (pnt_x - x) ** 2 + (pnt_y - y) ** 2 <= rad ** 2 \
                    and self.is_in_mask(img_red[y, x]) and self.is_in_mask(img_nir[y, x]):
                        n += 1
                        subj = float(-line_a) * img_red[y, x] - line_b + img_nir[y, x]
                        res_x += x * subj
                        res_y += y * subj
                        ave_x += x
                        ave_y += y
                        ave_subj += subj
        return {'xx': (res_x - ave_x * ave_subj) / float(n) if n > 0 else -9999,
               'yy': (res_y - ave_y * ave_subj) / float(n) if n > 0 else -9999,
               'avg': ave_subj / float(n) if n > 0 else -9999,
               'n': n}
    
    def brat_first(self):
        json_f1 = self.json['brat_first']
        imgs = self.read_images('in') # читать не отсюда
        cdef np.ndarray[np.uint16_t, ndim = 2] img_red = imgs['red'].astype('uint16', copy = False)
        cdef np.ndarray[np.uint16_t, ndim = 2] img_nir = imgs['nir'].astype('uint16', copy = False)
        cdef int rad = json_f1['rad'], x_pts = json_f1['xPts'], y_pts = json_f1['yPts']
        cdef int line_a, line_b
        line_a, line_b = self.get_soil_line(imgs)
        cdef int x_abs, y_abs, i, j, x, y, x_abs_0 = json_f1['xAbs0'], y_abs_0 = json_f1['yAbs0']
        cdef int x_abs_step = json_f1['xAbsStep'],  y_abs_step = json_f1['yAbsStep']
        cdef int need_comma = 0
        
        f = open(os.path.join(self.json['path'], self.json['paths']['bratTxt'], self.path + \
                              self.json['suffs']['bratTxt']), 'w')
        for i in range(x_pts):
            for j in range(y_pts):
                x = int(i / (x_pts - 1.0) * img_red.shape[1])
                y = int(j / (y_pts - 1.0) * img_red.shape[0])
                x_abs = x_abs_0 + x * x_abs_step
                y_abs = y_abs_0 + y * y_abs_step
                vec = self.brat_vec(img_red, img_nir, x, y, rad, line_a, line_b)
                if need_comma:
                    f.write(',')
                f.write('{{%i,%i},{%i,%i},{%f,%f},%f,%i}\n' % 
                        (x, y, x_abs, y_abs, float(vec['xx']), float(vec['yy']), float(vec['avg']), vec['n']))
                if not need_comma:
                    need_comma = 1
        f.write('}')
        f.close()
        
cdef class SceneGroup:
    cdef object json
    
    def __init__(self, config_file='1.json'):
        with open(config_file) as data_file:    
            self.json = simplejson.load(data_file)
    
    cdef make_axy_from_all(self, np.ndarray[np.float, ndim=2] imgs):
        cdef int i, n 
        cdef np.ndarray[np.float, ndim=1] reds, nirs
        cdef np.ndarray[np.float, ndim=2] axy = np.ones((imgs.shape[0], 3)) * -99
        n = imgs.shape[1] / 2
        for i in range(imgs.shape[0]):
            reds = imgs[i, :n]
            reds = reds[reds != -99]
            nirs = imgs[i, n:]
            nirs = nirs[nirs != -99]
            if len(reds) > 5:
                axy[i, 0] = np.linalg.lstsq(np.vstack([reds, np.ones(len(reds))]).T, nirs)[0][0]
                axy[i, 1] = np.mean(reds) + np.mean(nirs)
                axy[i, 2] = np.mean(reds) - np.mean(nirs)
        return axy
    
    def make_X_from_coords(self, coords):
        n = len(self.json['scenes'])
        imgs = np.empty((len(coords), n * 2), dtype=np.float)
        for i in range(n):
            S = Scene(i)
            img_red, img_nir = S.read_images('solp') 
            img_red, img_nir = S.normalize_imgs(img_red, img_nir) ### ^^^
            imgs[:, i] = img_red[coords[:, 0], coords[:, 1]]
            imgs[:, n + i] = img_nir[coords[:, 0], coords[:, 1]]
        return self.make_axy_from_all(imgs) ### <<<
    

In [6]:
for i in range(5, 6):
    A = Scene(i)
    print 'calculating', i, 'scene...'
    # imgs = A.read_images('solp')
    print A.get_soil_line(imgs)
    plot_array = A.get_plot_array(imgs)
    # A.save_plot_array(plot_array)    # Эти 3 строчки для рисования графика
    # A.make_solp_scene()    # Эта строчка для фильтрации снимков по солпу с обведённых графиков
    # A.brat_first()    # Эта строчка для генерации txt брату

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-6-292d86ad616c>, line 3)

In [156]:
### Paint .png and .tif maps
import numpy as np
import simplejson
import matplotlib.pyplot as plt
%matplotlib inline

config_file = '2.json'
with open(config_file) as data_file:    
    json = simplejson.load(data_file)
height, width = 100, 100 # json['size']['height'], json['size']['width']
coords = np.array(np.meshgrid(range(height), range(width))).T.reshape(height * width, 2)
X = SceneGroup(config_file).make_X_from_coords(coords) # .reshape((height, width, 3))

arr_c = np.genfromtxt(json['docs']['c'])
arr_6 = np.genfromtxt(json['docs']['6'])
X -= arr_6[:, 0]
X /= arr_6[:, 1]
ans_c = np.zeros((X.shape[0],), dtype=np.int)
min_dist = np.ones((X.shape[0],), dtype=np.int) * 100500
for i, c in enumerate(arr_c):
    dist = (X - c) ** 2
    ans_c[min_dist > dist] = i + 1
    min_dist[min_dist > dist] = dist
ans_c[X[:, 0] == -99] = 0
matplotlib.image.imsave('pixel_map.png', ans_c)
skimage.io.imsave('pixel_map.tif', ans_c, plugin = 'tifffile')

KeyError: 'docs'

In [140]:
import os
in_path = '/home/fila/Downloads/2bands5/'
out_path = '/home/fila/Desktop/landsat2/solp/'
names = []
for name_full in os.listdir(in_path):
    name = name_full[:name_full.find('_')]
    names.append(name)
    img = skimage.io.imread(in_path + name_full)
    skimage.io.imsave(out_path + 'red/' + name + '.tif', img[:, :, 0], plugin='tifffile')
    skimage.io.imsave(out_path + 'nir/' + name + '.tif', img[:, :, 1], plugin='tifffile')
print(len(names))

35


In [152]:
for i, name in enumerate(sorted(names)):
    bit = 8
    if (name[2] == '8'):
        bit = 16
    print('{\n"bit": "' + str(bit) + '",\n"id": "' +  str(i) + '",\n"path": "'+ name + '"\n' + '},')

{
"bit": "16",
"id": "0",
"path": "lc81780232014088lgn00"
},
{
"bit": "16",
"id": "1",
"path": "lc81780232014232lgn00"
},
{
"bit": "16",
"id": "2",
"path": "lc81780232014264lgn00"
},
{
"bit": "16",
"id": "3",
"path": "lc81780232014296lgn00"
},
{
"bit": "16",
"id": "4",
"path": "lc81780232015075lgn00"
},
{
"bit": "16",
"id": "5",
"path": "lc81780232015187lgn00"
},
{
"bit": "8",
"id": "6",
"path": "le71780231999279edc00"
},
{
"bit": "8",
"id": "7",
"path": "le71780232000138sgs00"
},
{
"bit": "8",
"id": "8",
"path": "le71780232001124sgs01"
},
{
"bit": "8",
"id": "9",
"path": "le71780232002143sgs00"
},
{
"bit": "8",
"id": "10",
"path": "le71780232002239sgs01"
},
{
"bit": "8",
"id": "11",
"path": "le71780232002287sgs01"
},
{
"bit": "8",
"id": "12",
"path": "le71780232003146asn00"
},
{
"bit": "8",
"id": "13",
"path": "lt51780231985136kis00"
},
{
"bit": "8",
"id": "14",
"path": "lt51780231985216xxx03"
},
{
"bit": "8",
"id": "15",
"path": "lt51780231986123xxx02"
},
{
"bit": "8",
"id": "16",
"p

In [ ]:
"height": 2132,
"width": 2755


In [100]:
aaa = np.array([[0, 0, 2], [0, 1, 2], [1, 0, 2], [1, 1, 2]])
aaa.reshape((2, 2, 3))

array([[[0, 0, 2],
        [0, 1, 2]],

       [[1, 0, 2],
        [1, 1, 2]]])

In [101]:
np.vectorize(lambda x: x * 2)(aaa)

array([[0, 0, 4],
       [0, 2, 4],
       [2, 0, 4],
       [2, 2, 4]])

In [90]:
aaa[[0, 1, 2], [1, 2, 2]]

array([0, 4, 4])

In [118]:
bbb = np.arange(12).reshape((2, 2, 3))
print(bbb)
bbb[[0, 1], [0, 1]]

[[[ 0  1  2]
  [ 3  4  5]]

 [[ 6  7  8]
  [ 9 10 11]]]


array([[ 0,  1,  2],
       [ 9, 10, 11]])

In [116]:
ccc = np.arange(6).reshape((2, 3))
ccc += 1
ddd = np.empty((2, 3))
np.dstack((ddd, ccc))

array([[[  4.94065646e-324,   1.00000000e+000],
        [  9.88131292e-324,   2.00000000e+000],
        [  1.48219694e-323,   3.00000000e+000]],

       [[  1.97626258e-323,   4.00000000e+000],
        [  2.47032823e-323,   5.00000000e+000],
        [  2.96439388e-323,   6.00000000e+000]]])

In [113]:
ccc

array([[1, 2, 3],
       [4, 5, 6]])